In [22]:
import hddm
import pandas as pd
import numpy as np
from patsy import dmatrix

In [2]:
# subject numbers
n_subjects = 10

# trial numbers
trials_per_level = 150

In [3]:
level1_a = {'v':0.5, 'a':2, 't':0.3, 'z':0.7, 'sv':0, 'sz':0, 'st':0} 

level2_a = {'v':0.5, 'a':2, 't':0.3, 'z':0.5, 'sv':0, 'sz':0, 'st':0} 

level3_a = {'v':0.2, 'a':2, 't':0.3, 'z':0.7, 'sv':0, 'sz':0, 'st':0} 

level4_a = {'v':0.2, 'a':2, 't':0.3, 'z':0.5, 'sv':0, 'sz':0, 'st':0} 

In [4]:
data_a, params_a = hddm.generate.gen_rand_data({
    'level1':level1_a,
    'level2':level2_a,
    'level3':level3_a,
    'level4':level4_a},
    size = trials_per_level,
    subjs = n_subjects
    ) 

In [5]:
level1_b = {'v':0.5, 'a':2, 't':0.3, 'z':0.7, 'sv':0, 'sz':0, 'st':0} 

level2_b = {'v':0.5, 'a':2, 't':0.3, 'z':0.5, 'sv':0, 'sz':0, 'st':0} 

level3_b = {'v':0.2, 'a':2, 't':0.3, 'z':0.7, 'sv':0, 'sz':0, 'st':0} 

level4_b = {'v':0.2, 'a':2, 't':0.3, 'z':0.5, 'sv':0, 'sz':0, 'st':0} 

In [6]:
data_b, params_b = hddm.generate.gen_rand_data({
    'level1':level1_b,
    'level2':level2_b,
    'level3':level3_b,
    'level4':level4_b},
    size = trials_per_level,
    subjs = n_subjects
    ) 

In [7]:
data_a['stimulus'] = pd.Series(np.ones((len(data_a)))*0, index = data_a.index)
data_b['stimulus'] = pd.Series(np.ones((len(data_b))), index = data_b.index)

data_a['coherence'] = pd.Series(np.hstack((np.ones(int(len(data_a)/2))*1,np.ones(int(len(data_a)/2))*0)), index = data_a.index)
data_b['coherence'] = pd.Series(np.hstack((np.ones(int(len(data_a)/2))*1,np.ones(int(len(data_a)/2))*0)), index = data_b.index)

data_a['attention'] = pd.Series(np.hstack((np.ones(int(len(data_a)/4))*1,np.ones(int(len(data_a)/4))*0,np.ones(int(len(data_a)/4))*1,np.ones(int(len(data_a)/4))*0)), index = data_a.index)
data_b['attention'] = pd.Series(np.hstack((np.ones(int(len(data_a)/4))*1,np.ones(int(len(data_a)/4))*0,np.ones(int(len(data_a)/4))*1,np.ones(int(len(data_a)/4))*0)), index = data_b.index)

# merge data for stimulus A(Car) and B(Face)
data = data_a.append(data_b, ignore_index = True)

/tmp/ipykernel_11419/3847642742.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_a.append(data_b, ignore_index = True)


In [8]:
rawdata = data

In [46]:
def z_link_func(x, data=data):
        stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                                  {'s': data.stimulus.loc[x.index]},return_type='dataframe'))
        )
        # Apply z = (1 - x) to flip them along 0.5
        z_flip = np.subtract(stim, x.to_frame())
        # The above inverts those values we do not want to flip,
        # so invert them back
        z_flip[stim == 0] *= -1
        return z_flip

In [33]:
# The link function for ``z`` is:
z_reg = {'model': "z ~ 1 + C(condition)", 'link_func': z_link_func}
# The link function for ``v`` is:
v_reg = {'model': "v ~ 1 + C(condition)", 'link_func': lambda x:x}
# Then we can put the regression description for the hddm model.
reg_descr = [z_reg, v_reg]
# Finally we run the regression model 
m_reg4 = hddm.HDDMRegressor(data, 
                            reg_descr,
                            group_only_regressors=True,
                            include=['z'],       
                            p_outlier=.05)

Adding these covariates:
['z_Intercept', 'z_C(condition)[T.level2]', 'z_C(condition)[T.level3]', 'z_C(condition)[T.level4]']
Adding these covariates:
['v_Intercept', 'v_C(condition)[T.level2]', 'v_C(condition)[T.level3]', 'v_C(condition)[T.level4]']


In [40]:
m_reg4.model_descrs

[{'outcome': 'z',
  'model': ' 1 + C(condition)',
  'params': ['z_Intercept',
   'z_C(condition)[T.level2]',
   'z_C(condition)[T.level3]',
   'z_C(condition)[T.level4]'],
  'link_func': <function __main__.z_link_func(x, data=             rt  response  subj_idx condition  stimulus  coherence  attention
0      0.902376       1.0         0    level1       0.0        1.0        1.0
1      1.050672       1.0         0    level1       0.0        1.0        1.0
2      1.156124       1.0         0    level1       0.0        1.0        1.0
3      1.594156       1.0         0    level1       0.0        1.0        1.0
4      0.526943       1.0         0    level1       0.0        1.0        1.0
...         ...       ...       ...       ...       ...        ...        ...
11995  0.684454       1.0         9    level4       1.0        0.0        0.0
11996  1.726236       0.0         9    level4       1.0        0.0        0.0
11997  2.291932       1.0         9    level4       1.0        0.0      

In [41]:
m_reg4.nodes_db.to_csv('test.csv')

In [42]:
m_reg4.get_non_observeds().to_csv("test2.csv")

In [43]:
reg = m_reg4.model_descrs[0]
reg

{'outcome': 'z',
 'model': ' 1 + C(condition)',
 'params': ['z_Intercept',
  'z_C(condition)[T.level2]',
  'z_C(condition)[T.level3]',
  'z_C(condition)[T.level4]'],
 'link_func': <function __main__.z_link_func(x, data=             rt  response  subj_idx condition  stimulus  coherence  attention
0      0.902376       1.0         0    level1       0.0        1.0        1.0
1      1.050672       1.0         0    level1       0.0        1.0        1.0
2      1.156124       1.0         0    level1       0.0        1.0        1.0
3      1.594156       1.0         0    level1       0.0        1.0        1.0
4      0.526943       1.0         0    level1       0.0        1.0        1.0
...         ...       ...       ...       ...       ...        ...        ...
11995  0.684454       1.0         9    level4       1.0        0.0        0.0
11996  1.726236       0.0         9    level4       1.0        0.0        0.0
11997  2.291932       1.0         9    level4       1.0        0.0        0.0
1

In [50]:
args=[]
for arg in reg['params']:
    args.append(arg)
params = np.matrix(args)
params

matrix([['z_Intercept', 'z_C(condition)[T.level2]',
         'z_C(condition)[T.level3]', 'z_C(condition)[T.level4]']],
       dtype='<U24')

In [56]:
design_matrix = dmatrix("C(attention)*C(coherence)",data,return_type='dataframe',NA_action='raise')
design_matrix = design_matrix.loc[data.index]
design_matrix

,Intercept,C(attention)[T.1.0],C(coherence)[T.1.0],C(attention)[T.1.0]:C(coherence)[T.1.0]
0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0
...,...,...,...,...
11995,1.0,0.0,0.0,0.0
11996,1.0,0.0,0.0,0.0
11997,1.0,0.0,0.0,0.0
11998,1.0,0.0,0.0,0.0


In [54]:
params.T

matrix([['z_Intercept'],
        ['z_C(condition)[T.level2]'],
        ['z_C(condition)[T.level3]'],
        ['z_C(condition)[T.level4]']], dtype='<U24')

In [55]:
design_matrix.dot(params.T)

ValueError: data type must provide an itemsize